In [8]:
!pip install -qU llama-index-llms-huggingface llama-index-llms-huggingface-api

In [2]:
from llama_index.core.llama_dataset import download_llama_dataset

In [3]:
import os
from getpass import getpass

hf_token = getpass()

In [4]:
from huggingface_hub import login

login(token=hf_token, add_to_git_credential=True)

/home/mahbub/Documents/dev/llama_index_projects/.conda/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Token is valid (permission: fineGrained).
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in case you want to set the 'store' credential helper as default.

git config --global credential.helper store

Read https://git-scm.com/book/en/v2/Git-Tools-Credential-Storage for more details.
Token has not been saved to git credential helper.
Your token has been saved to /home/mahbub/.cache/huggingface/token
Login successful


In [5]:
rag_dataset, documents = download_llama_dataset("BlockchainSolanaDataset", "./data")

In [6]:
rag_dataset.to_pandas()[:3]

,query,reference_contexts,reference_answer,reference_answer_by,query_by
0,What are the key issues preventing the wide ad...,[From Bitcoin to Solana – Innovating Blockchai...,The key issues preventing the wide adoption of...,ai (gpt-3.5-turbo),ai (gpt-3.5-turbo)
1,How does blockchain technology provide data se...,[From Bitcoin to Solana – Innovating Blockchai...,Blockchain technology provides data security a...,ai (gpt-3.5-turbo),ai (gpt-3.5-turbo)
2,What is the projected growth rate of the block...,[2 \n \nchain market size is expected to grow ...,The projected growth rate of the blockchain ma...,ai (gpt-3.5-turbo),ai (gpt-3.5-turbo)


In [9]:
from llama_index.llms.ollama import Ollama
from llama_index.embeddings.ollama import OllamaEmbedding
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.llms.huggingface_api import HuggingFaceInferenceAPI

ollama_llm = Ollama(model="phi3", request_timeout=60.0, temperature=0)
hf_llm = HuggingFaceInferenceAPI(model_name="google/gemma-2-2b-it", token=hf_token)

In [10]:
from llama_index.core import VectorStoreIndex

embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")

index = VectorStoreIndex.from_documents(documents=documents, embed_model = embed_model)

In [11]:
#query engine

query_engine = index.as_query_engine(llm=ollama_llm)
query_engine

In [ ]:
import tqdm
from llama_index.core.llama_pack import download_llama_pack

RagEvaluatorPack = download_llama_pack("RagEvaluatorPack", "./rag_evaluator_pack")

rag_evaluator_pack = RagEvaluatorPack(rag_dataset=rag_dataset, embed_model='local', query_engine=query_engine, judge_llm=ollama_llm)

In [13]:
from llama_index.core.evaluation import FaithfulnessEvaluator
evaluator_model = FaithfulnessEvaluator(llm=ollama_llm)

In [14]:
import nest_asyncio
nest_asyncio.apply()

In [15]:
response_vector = query_engine.query("What is the first thing I need to do when I have covid symptoms?")

In [16]:
print(response_vector)

Seek medical attention as soon as possible if you experience fever, cough, and difficulty breathing due to COVID-19 symptoms. It's important to call ahead by telephone before visiting a healthcare facility for guidance on the right place of care based on your local situation.


In [17]:
eval_result = evaluator_model.evaluate_response(response=response_vector)

In [18]:
print(eval_result.response)

Seek medical attention as soon as possible if you experience fever, cough, and difficulty breathing due to COVID-19 symptoms. It's important to call ahead by telephone before visiting a healthcare facility for guidance on the right place of care based on your local situation.
